<h1>Bases des télécommunications 1</h1>

**__Introduction__**

Le but de ce laboratoire est de mettre en pratique les concepts vus en classe de Bases des
Télécommunications. On travaillera ainsi sur la compression d’un fichier texte

- Le rendu du **jupyter notebook** doit se faire sur __cyberlearn__ avec comme nom "prenom_nom_groupe" 
- Le fichier texte utilisé ici doit être encodé en UTF-8 (sans BOM), vous pouvez le vérifier avec Notepad++ par exemple.
- Exemple de code python 3 pour ouvrir en lecture un fichier texte en UTF-8 : __*open(INPUT, "r", encoding="UTF-8")*__

<h2>I. Tris alphabétique + fréquentiel</h2>

a)  Réaliser  ci-dessous un programme qui lit un fichier texte (input.txt), puis en écrit un nouveau (out1.txt)
en plaçant tous les caractères dans l’ordre alphabétique (ordre de leur valeur
Ascii).
Exemple : **abb@acA → @Aaabbc** (La fonction sorted de Python 3 pourrait vous aider, tout comme l’utilisation
de dictionnaires (voir tuto_python.ipynb))



In [1]:
# exercice a.
import math

input_file = open("input.txt", "r", encoding="UTF-8")
out_file = open("out1.txt", "w+")

file_text = input_file.read()
out_text = ""
ascii_text = []

for character in file_text:
    ascii_text.append(ord(character))

ascii_text.sort()

for acii_character in ascii_text:
    out_text += chr(acii_character)

out_file.write(out_text)

404

b) Ajouter à votre premier programme une écriture d’un nouveau fichier texte
(out2.txt), mais cette fois trié dans l’ordre des fréquences d’apparition.
**Exemple : abb@acA → aabb@Ac**
Votre programme doit afficher dans la console, la fréquence d’apparition pour
chaque caractère. (Ainsi que la valeur ascii du caractère)

Exemple : 

<table>
<tr>
    <td>0x61</td>
    <td>0.285</td>
</tr>
<tr>
    <td>0x62</td>
    <td>0.285</td>
</tr>
<tr>
    <td>0x41</td>
    <td>0.142</td>
</tr>
</table>



In [2]:
# exercice b.
out_file2 = open("out2.txt", "w+")

char_dictionnary = {}
sum_char = len(file_text)
out_text2 = ""

for character in file_text:
    if character in char_dictionnary:
        char_dictionnary[character] += 1
    else:
        char_dictionnary[character] = 1

char_dictionnary = dict(sorted(char_dictionnary.items(), key=lambda item: item[1], reverse=True))

for key in char_dictionnary:
    print("0x{} {:.10f}".format(ord(key), (char_dictionnary[key]) / sum_char))


for key in char_dictionnary:
    out_text2 += ''.join([char*char_dictionnary[key] for char in key]) 

out_file2.write(out_text2)

#Calculer entropie H
H = 0
for key in char_dictionnary:
    prob = char_dictionnary[key] / sum_char
    H += prob * math.log2(prob)
    
H = H * -1
print("Entropie du fichier H = {:.5f}".format(H))

#La quantité de décision D?
D = math.log2(len(char_dictionnary))
print("Quantité de décision D = {:.5f}".format(D))

#La redondance R
R = D-H
print("Redondance R = {:.5f}".format(R))

#Taux de compression maximum du fichier
L = 1 - H / D

print("Taux de compression maximum du fichier = {:.5f}".format(L))    


0x32 0.1509900990
0x101 0.1212871287
0x116 0.0841584158
0x117 0.0618811881
0x115 0.0594059406
0x97 0.0569306931
0x110 0.0569306931
0x114 0.0495049505
0x105 0.0470297030
0x111 0.0420792079
0x108 0.0371287129
0x100 0.0297029703
0x109 0.0297029703
0x112 0.0222772277
0x99 0.0173267327
0x46 0.0173267327
0x39 0.0123762376
0x120 0.0123762376
0x233 0.0123762376
0x118 0.0099009901
0x10 0.0099009901
0x76 0.0074257426
0x73 0.0074257426
0x113 0.0074257426
0x68 0.0074257426
0x98 0.0074257426
0x103 0.0049504950
0x67 0.0049504950
0x232 0.0024752475
0x102 0.0024752475
0x231 0.0024752475
0x224 0.0024752475
0x44 0.0024752475
Entropie du fichier H = 4.27256
Quantité de décision D = 5.04439
Redondance R = 0.77183
Taux de compression maximum du fichier = 0.15301


c) Améliorer votre programme pour qu’il calcule l’entropie, la quantité de décision,
la redondance et le taux de compression maximal du fichier. Le tout est à afficher
dans la console.

**N’oubliez pas que l’espace, la ponctuation et le retour chariot sont aussi considéré comme des
caractères. Il est donc juste qu’ils soient pris en compte par votre programme.**


<h2>II. Code de Huffman</h2>

On veut construire un code de Huffman pour compresser notre fichier texte. On part du principe
que le code n’a pas besoin d’être transmis dans le fichier compressé. 


a) À partir de l’exercice précédent, écrire un code générant un codage de Huffman pour
les caractères de ce fichier. Votre programme doit afficher la valeur ASCII des
charactères et leur code binaire généré par codage de Huffman.

Exemple:

<table>
<tr>
    <td>0x61</td>
    <td>11</td>
</tr>
<tr>
    <td>0x62</td>
    <td>10</td>
</tr>
<tr>
    <td>0x40</td>
    <td>00</td>
</tr>
<tr>
    <td>0x63</td>
    <td>011</td>
</tr>
<tr>
    <td>0x41</td>
    <td>010</td>
</tr>
</table>

In [3]:
#exercice a.
output_text = ""
huffman_done = []
char_dict_cpy = char_dictionnary.copy()
dict_huffman = {}
count_char = 0

def get_min_value_key(dict_huffman):
    min_value = 255
    min_key = ''
    for key in dict_huffman:
        if dict_huffman[key] < min_value and key not in huffman_done:
            min_value = dict_huffman[key]
            min_key = key

    huffman_done.append(min_key)
    return min_key

while count_char < len(char_dictionnary)-1:
    # Prends les deux clés les plus petites
    min_key_0 = get_min_value_key(char_dict_cpy)
    min_key_1 = get_min_value_key(char_dict_cpy)

    for char_key_0 in min_key_0:
        if char_key_0 in dict_huffman:
            dict_huffman[char_key_0] = "0" + dict_huffman[char_key_0]
        else:
            dict_huffman[char_key_0] = "0"
    
    for char_key_1 in min_key_1:
        if char_key_1 in dict_huffman:
            dict_huffman[char_key_1] = "1" + dict_huffman[char_key_1]
        else:
            dict_huffman[char_key_1] = "1"

    count_char += 1
    char_dict_cpy[min_key_0+min_key_1] = char_dict_cpy[min_key_0] + char_dict_cpy[min_key_1]

## Affichage
for key in char_dictionnary:
    print("0x{} {}".format(ord(key), dict_huffman[key]))


0x32 110
0x101 100
0x116 1111
0x117 1010
0x115 0110
0x97 0100
0x110 0101
0x114 0010
0x105 0000
0x111 11101
0x108 10111
0x100 00111
0x109 01110
0x112 00010
0x99 101100
0x46 101101
0x39 000110
0x120 000111
0x233 001100
0x118 1110001
0x10 1110010
0x76 0011011
0x73 0111100
0x113 0111101
0x68 0111110
0x98 0111111
0x103 11100001
0x67 11100110
0x232 111001110
0x102 111001111
0x231 00110100
0x224 00110101
0x44 11100000


b) Afficher le taux moyen de bits par caractère de votre code, dans la console.

In [4]:
## Affichage
sum_bits = 0
for key in char_dictionnary:
    sum_bits += len(dict_huffman[key]) * char_dictionnary[key]

print("{}".format(sum_bits / sum_char))


input_file.close()
out_file.close()
out_file2.close()

4.297029702970297


 c) Que pouvez-vous en déduire par rapport à l’entropie du point (I.c), en une phrase ?
 
Le taux moyen de bits est plus élevé que l'entropie. Plus le taux se rapproche de l'entropie, meilleur est l'algorithme.


**TODO**

d) Compresser le fichier texte d’input dans un nouveau fichier binaire en utilisant votre
code. Si votre message codé n’est pas un multiple de 8, rajouter des bits 0 à la fin. Le
fichier compressé doit avoir une taille sur disque significativement plus petite que
celui de base.

e) Que manquerait-il a ce fichier binaire pour pouvoir être décodé par n’importe qui ?
(Pas besoin de l’implémenter pour ce labo)

La talbe d'équivalence entre les valeurs en bits et en ascii.

**TODO**

# Bonus I

Coder la fonction de tri avec **DES BOUCLES ET DES LISTES**

In [ ]:

tabOfChars=['b','a','a','f'] # transformation du fichier lu en tableau de charactères
mySortedList= tabOfChars.copy()
def mySortedFunc(mySortedList):#insérer vos arguments
    return mySortedList


In [ ]:
assert sorted(tabOfChars) == mySortedFunc(mySortedList)

# Bonus II - Shannon-Fano

On veut construire un code de Shannon-Fano pour compresser notre fichier texte. On part du
principe que le code n’a pas besoin d’être transmis dans le fichier compressé.

a) Réaliser ci-dessous un programme qui lit un fichier texte, puis génère un codage de
Shannon-Fano pour les caractères de ce fichier. Votre programme doit afficher le code
binaire choisi pour chaque caractère.

Exemple :

<table>
<tr>
    <td>a</td>
    <td>0x61</td>
    <td>11</td>
</tr>
<tr>
    <td>b</td>
    <td>0x62</td>
    <td>10</td>
</tr>
<tr>
    <td>@</td>
    <td>0x40</td>
    <td>00</td>
</tr>
<tr>
    <td>c</td>
    <td>0x63</td>
    <td>011</td>
</tr>
<tr>
    <td>A</td>
    <td>0x41</td>
    <td>010</td>
</tr>
</table>

In [1]:
# exercice a.fjh

b) Afficher le taux moyen de bits par caractère de votre code, dans la console.

In [ ]:
# exercice b.

 c) Que pouvez-vous en déduire par rapport à l’entropie du point (I.c), en une phrase ?

**TODO**

d) Compresser le fichier texte d’input dans un nouveau fichier binaire en utilisant votre code. Si
votre message codé n’est pas un multiple de 8, rajouter des 0 à la fin. Le fichier
compressé doit avoir une taille sur disque significativement plus petite que celui de
base